In [ ]:
import os, time, pickle, easydict, network, util
import tensorflow as tf
import numpy as np

opt= easydict.EasyDict({ "dataset": 'facades', 
                        "train_subfolder": 'train',
                        "test_subfolder": 'val',
                        "batch_size":1,
                        "test_batch_size": 2,
                        "ngf":64,
                        "ndf":64,
                        "input_size":256,
                        "crop_size":256,
                        "resize_scale":286,
                        "fliplr":True,
                        "train_epoch":20,
                        "lrD":0.0002,
                        "lrG":0.0002,
                        "L1_lambda":100,
                        "beta1":0.5,
                        "beta2":0.999,
                        "save_root":'results',
                        "inverse_order":True
                       })

root = opt.dataset + '_' + opt.save_root + '/'
model = opt.dataset + '_'
if not os.path.isdir(root):
    os.mkdir(root)
if not os.path.isdir(root + 'Fixed_results'):
    os.mkdir(root + 'Fixed_results')

train_loader = util.data_loader('D:/SEM and OM pix2pix data/data/' + opt.dataset + '/' + opt.train_subfolder, opt.batch_size, shuffle=True)
test_loader = util.data_loader('D:/SEM and OM pix2pix data/data/' + opt.dataset + '/' + opt.test_subfolder, opt.test_batch_size, shuffle=True)
img_size = train_loader.shape[1]
test_img = test_loader.next_batch()
if opt.inverse_order:
    fixed_x_ = test_img[:, :, img_size:, :]
    fixed_y_ = test_img[:, :, 0:img_size, :]
else:
    fixed_x_ = test_img[:, :, 0:img_size, :]
    fixed_y_ = test_img[:, :, img_size:, :]

if img_size != opt.input_size:
    fixed_x_ = util.imgs_resize(fixed_x_, opt.input_size)
    fixed_y_ = util.imgs_resize(fixed_y_, opt.input_size)

fixed_x_ = util.norm(fixed_x_)
fixed_y_ = util.norm(fixed_y_)

x = tf.placeholder(tf.float32, shape=(None, opt.input_size, opt.input_size, train_loader.shape[3]))
y = tf.placeholder(tf.float32, shape=(None, opt.input_size, opt.input_size, train_loader.shape[3]))

G = network.generator(x, opt.ngf)
D_positive, D_positive_logits = network.discriminator(x, y, opt.ndf)
D_negative, D_negative_logits = network.discriminator(x, G, opt.ndf, reuse=True)

D_loss_positive = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_positive_logits, labels=tf.ones_like(D_positive_logits)))
D_loss_negative = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_negative_logits, labels=tf.zeros_like(D_negative_logits)))
D_loss = (D_loss_positive + D_loss_negative) * 0.5
G_loss_gan = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_negative_logits, labels=tf.ones_like(D_negative_logits)))
G_loss_L1 = tf.reduce_mean(tf.reduce_sum(tf.abs(G - y), 3))
G_loss = G_loss_gan + opt.L1_lambda * G_loss_L1

T_vars = tf.trainable_variables()
D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
G_vars = [var for var in T_vars if var.name.startswith('generator')]

with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    D_optim = tf.train.AdamOptimizer(opt.lrD, beta1=opt.beta1, beta2=opt.beta2).minimize(D_loss, var_list=D_vars)
    G_optim = tf.train.AdamOptimizer(opt.lrG, beta1=opt.beta1, beta2=opt.beta2).minimize(G_loss, var_list=G_vars)

saver = tf.train.Saver()
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []

print('training start!')
start_time = time.time()
for epoch in range(opt.train_epoch):
    D_losses = []
    G_losses = []
    epoch_start_time = time.time()
    num_iter = 0
    for iter in range(train_loader.shape[0] // opt.batch_size):
        train_img = train_loader.next_batch()

        if opt.inverse_order:
            x_ = train_img[:, :, img_size:, :]
            y_ = train_img[:, :, 0:img_size, :]
        else:
            x_ = train_img[:, :, 0:img_size, :]
            y_ = train_img[:, :, img_size:, :]

        if img_size != opt.input_size:
            x_ = util.imgs_resize(x_, opt.input_size)
            y_ = util.imgs_resize(y_, opt.input_size)

        if opt.resize_scale:
            x_ = util.imgs_resize(x_, opt.resize_scale)
            y_ = util.imgs_resize(y_, opt.resize_scale)

        if opt.crop_size:
            x_, y_ = util.random_crop(x_, y_, opt.crop_size)

        if opt.fliplr:
            x_, y_ = util.random_fliplr(x_, y_)

        x_ = util.norm(x_)
        y_ = util.norm(y_)

        loss_d_, _ = sess.run([D_loss, D_optim], {x: x_, y: y_})
        D_losses.append(loss_d_)
        train_hist['D_losses'].append(loss_d_)

        loss_g_, _ = sess.run([G_loss, G_optim], {x: x_, y: y_})
        G_losses.append(loss_g_)
        train_hist['G_losses'].append(loss_g_)

        num_iter += 1

    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time

    print('[%d/%d] - ptime: %.2f, loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), opt.train_epoch, per_epoch_ptime, np.mean((D_losses)), np.mean(G_losses)))
    fixed_p = root + 'Fixed_results/' + model + str(epoch + 1) + '.png'
    outputs = sess.run(G, {x: fixed_x_})
    util.show_result(fixed_x_, outputs, fixed_y_, (epoch+1), save=True, path=fixed_p)
    train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

end_time = time.time()
total_ptime = end_time - start_time
train_hist['total_ptime'].append(total_ptime)

print("Avg. one epoch ptime: %.2f, total %d epochs ptime: %.2f" % (np.mean(train_hist['per_epoch_ptimes']), opt.train_epoch, total_ptime))
print("Training finish!... save training results")
with open(root + model + 'train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

saver.save(sess, root + model + 'params.ckpt')

util.show_train_hist(train_hist, save=True, path=root + model + 'train_hist.png')

#pix2pix_training_code

{'dataset': 'facades', 'train_subfolder': 'train', 'test_subfolder': 'val', 'batch_size': 1, 'test_batch_size': 2, 'ngf': 64, 'ndf': 64, 'input_size': 256, 'crop_size': 256, 'resize_scale': 286, 'fliplr': True, 'train_epoch': 20, 'lrD': 0.0002, 'lrG': 0.0002, 'L1_lambda': 100, 'beta1': 0.5, 'beta2': 0.999, 'save_root': 'results', 'inverse_order': True}


In [1]:
import os, time, easydict, network, util
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

opt= easydict.EasyDict({ "dataset": 'facades',
                        "input_size":256,
                        "test_subfolder": 'val',
                        "ngf":64,
                        "ndf":64,
                        "save_root":'results',
                        "inverse_order":True
                       })

if not os.path.isdir(opt.dataset + '_results/test_results'):
    os.mkdir(opt.dataset + '_results/test_results')

test_loader = util.data_loader('D:/SEM and OM pix2pix data/data/' + opt.dataset + '/' + opt.test_subfolder, 1, shuffle=False)
img_size = test_loader.shape[1]

x = tf.placeholder(tf.float32, shape=(None, opt.input_size, opt.input_size, test_loader.shape[3]))

G = network.generator(x, opt.ngf)

saver = tf.train.Saver()
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
saver.restore(sess, (opt.dataset + '_results/facades_params.ckpt'))

print('test start!')

per_ptime = []
total_start_time = time.time()
for iter in range(test_loader.shape[0]):
    per_start_time = time.time()

    train_img = test_loader.next_batch()

    if opt.inverse_order:
        x_ = train_img[:, :, img_size:, :]
        y_ = train_img[:, :, 0:img_size, :]
    else:
        x_ = train_img[:, :, 0:img_size, :]
        y_ = train_img[:, :, img_size:, :]

    if img_size != opt.input_size:
        x_ = util.imgs_resize(x_, opt.input_size)
        y_ = util.imgs_resize(y_, opt.input_size)

    x_ = util.norm(x_)
    y_ = util.norm(y_)

    test_img = sess.run(G, {x: x_})

    num_str = test_loader.file_list[iter][:test_loader.file_list[iter].find('.')]
    path = opt.dataset + '_results/test_results/' + num_str + '_input.png'
    plt.imsave(path, (util.denorm(x_[0]) / 255))
    path = opt.dataset + '_results/test_results/' + num_str + '_output.png'
    plt.imsave(path, (util.denorm(test_img[0]) / 255))
    path = opt.dataset + '_results/test_results/' + num_str + '_target.png'

    per_end_time = time.time()
    per_ptime.append(per_end_time - per_start_time)

total_end_time = time.time()
total_ptime = total_end_time - total_start_time

print('total %d images generation complete!' % (iter+1))
print('Avg. one image process ptime: %.2f, total %d images process ptime: %.2f' % (np.mean(per_ptime), (iter+1), total_ptime))

#pix2pix_test_code

{'dataset': 'facades', 'input_size': 256, 'test_subfolder': 'val', 'ngf': 64, 'ndf': 64, 'save_root': 'results', 'inverse_order': True}
INFO:tensorflow:Restoring parameters from facades_results/facades_params.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for facades_results/facades_params.ckpt
	 [[Node: save/RestoreV2_2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save/Const_0_0, save/RestoreV2_2/tensor_names, save/RestoreV2_2/shape_and_slices)]]
	 [[Node: save/RestoreV2_11/_29 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_200_save/RestoreV2_11", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'save/RestoreV2_2', defined at:
  File "C:\Users\32core\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\32core\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\32core\Anaconda3\envs\tensorflow\lib\site-packages\ipy